In [1]:
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap, jvp, jit
import jax
from functools import partial

a = 2 # 2aB
nH = 3
atoms = ""
for i in range(nH):
    atoms += f"H {i*a:.5f} 0.00000 0.00000 \n"

mol = gto.M(atom=atoms, basis="sto6g", unit='bohr', spin=1, verbose=4)
mol.build()

mf = scf.UHF(mol)
mf.kernel()

nfrozen = 0
mycc = cc.CCSD(mf,frozen=nfrozen)
mycc.kernel()[0]

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-34-generic', version='#34~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Tue Sep 23 15:35:20 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Wed Nov  5 16:06:15 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 3
[INPUT] num. electrons = 3
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = bohr
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000 Bohr   0.0
[INPUT]  2 H      1.058354421840   0.000000000000   0.0

np.float64(-0.029392467855749442)

In [2]:
eris = mycc.ao2mo(mycc.mo_coeff)
eccs = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
print('mf energy: ',mf.e_tot)
print('ccs energy: ', mf.e_tot+eccs)
# mycc.t1 = (mycc.t1[0]*10,mycc.t1[1]*10)
eccs = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
print('ccs energy (10 t1): ', mf.e_tot+eccs)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
print('ccsd energy (10 t1): ', mf.e_tot+eccsd)

mf energy:  -1.5439139532327748
ccs energy:  -1.5441982437908104
ccs energy (10 t1):  -1.5441982437908104
ccsd energy (10 t1):  -1.5733064210885241


In [3]:
options = {'n_eql': 4,
           'n_prop_steps': 50,
            'n_ene_blocks': 20,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 3,
            'seed': 2,
            'walker_type': 'uhf',
            'trial': 'uccsd_pt2_ad',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

from ad_afqmc import config
from ad_afqmc.prop_unrestricted import prop_unrestricted
import time
from jax import random
# from ad_afqmc.ccsd_pt import ccsd_pt, sample_ccsd_pt
# t1 = 5 * mycc.t1
# mycc.t1 = [10*mycc.t1[0],10*mycc.t1[1]]
# ccsd_pt.prep_afqmc(mycc,chol_cut=1e-7)
prop_unrestricted.prep_afqmc(mycc,options,chol_cut=1e-8)
ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ \
    = (prop_unrestricted._prep_afqmc(options))


#
# Preparing AFQMC calculation
# If you import pyscf cc modules and use MPI for AFQMC in the same script, finalize MPI before calling the AFQMC driver.
# Calculating Cholesky integrals
# Finished calculating Cholesky integrals
#
# Size of the correlation space:
# Number of electrons: (2, 1)
# Number of basis functions: 3
# Number of Cholesky vectors: 6
#
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: 3
# nelec: (2, 1)
#
# n_eql: 4
# n_prop_steps: 50
# n_ene_blocks: 20
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 3
# seed: 2
# walker_type: uhf
# trial: uccsd_pt2_ad
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
# LNO: False
# orbE: 0
# maxError: 0.001
#


In [4]:
config.setup_jax()
MPI = config.setup_comm()
init = time.time()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
seed = options["seed"]
neql = options["n_eql"]

trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1

ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
ham_data = ham.build_propagation_intermediates(
    ham_data, prop, trial, wave_data
)
prop_data = prop.init_prop_data(trial, wave_data, ham_data, None)

prop_data["key"] = random.PRNGKey(seed + rank)
print('init walker energy: ', prop_data['e_estimate'])
print('mf enegry: ', mf.e_tot)
walker_up = prop_data['walkers'][0][0]
walker_dn = prop_data['walkers'][1][0]
# et1 = jnp.real(trial._calc_energy(
#     walker_up, walker_dn, ham_data, wave_data
#     ))
# print('exact T1 transformed init walker energy: ', et1)
et1 = trial._calc_energy(
    wave_data['mo_ta'],wave_data['mo_tb'],ham_data,wave_data)
print('exact T1 init walker energy: ', et1)
eris = mycc.ao2mo(mycc.mo_coeff)
eccsd = mycc.energy(mycc.t1, (0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
print('ccs energy: ', mf.e_tot+eccsd)
print('ccsd energy: ', mycc.e_tot)

# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
init walker energy:  -1.5439139477596797
mf enegry:  -1.5439139532327748
exact T1 init walker energy:  -1.544198236915559
ccs energy:  -1.5441982437908104
ccsd energy:  -1.5733064210885241


In [5]:
t2aa = jnp.array(mycc.t2[0])
t2ab = jnp.array(mycc.t2[1])
t2bb = jnp.array(mycc.t2[2])
t2aa = (t2aa - t2aa.transpose(0, 1, 3, 2)) / 2
t2bb = (t2bb - t2bb.transpose(0, 1, 3, 2)) / 2
wave_data["t2aa"] = t2aa.transpose(0, 2, 1, 3)
wave_data["t2ab"] = t2ab.transpose(0, 2, 1, 3)
wave_data["t2bb"] = t2bb.transpose(0, 2, 1, 3)
wave_data["t1a"] = mycc.t1[0]
wave_data["t1b"] = mycc.t1[1]

In [6]:
from jax import lax
@partial(jit, static_argnums=0)
def _calc_energy_pt(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> complex:
    nocc_a, t2_aa = trial.nelec[0], wave_data["t2aa"]
    nocc_b, t2_bb = trial.nelec[1], wave_data["t2bb"]
    t2_ab = wave_data["t2ab"]
    mo_a, mo_b = wave_data['mo_ta'], wave_data['mo_tb']
    chol_a = ham_data["chol"][0].reshape(-1, trial.norb, trial.norb)
    chol_b = ham_data["chol"][1].reshape(-1, trial.norb, trial.norb)
    h1_a = ham_data["h1"][0]
    h1_b = ham_data["h1"][1]

    # full green's function G_pq
    green_a = (walker_up @ (jnp.linalg.inv(mo_a.T @ walker_up)) @ mo_a.T).T
    green_b = (walker_dn @ (jnp.linalg.inv(mo_b.T @ walker_dn)) @ mo_b.T).T
    greenp_a = (green_a - jnp.eye(trial.norb))[:,nocc_a:]
    greenp_b = (green_b - jnp.eye(trial.norb))[:,nocc_b:]

    hg_a = jnp.einsum("pq,pq->", h1_a, green_a)
    hg_b = jnp.einsum("pq,pq->", h1_b, green_b)
    hg = hg_a + hg_b # <exp(T1)HF|h1|walker>/<exp(T1)HF|walker>

    # 0 body energy
    # h0 = ham_data["h0"]

    # <exp(T1)HF|h1|walker>/<exp(T1)HF|walker>
    # one body energy
    e1_0 = hg

    # <exp(T1)HF|h2|walker>/<exp(T1)HF|walker>
    # two body energy
    lg_a = jnp.einsum("gpq,pq->g", chol_a, green_a, optimize="optimal")
    lg_b = jnp.einsum("gpq,pq->g", chol_b, green_b, optimize="optimal")
    e2_0_1 = ((lg_a + lg_b) @ (lg_a + lg_b)) / 2.0
    lg1_a = jnp.einsum("gpj,qj->gpq", chol_a, green_a, optimize="optimal")
    lg1_b = jnp.einsum("gpj,qj->gpq", chol_b, green_b, optimize="optimal")
    e2_0_2 = (
        -(
            jnp.sum(vmap(lambda x: x * x.T)(lg1_a))
            + jnp.sum(vmap(lambda x: x * x.T)(lg1_b))
        )
        / 2.0
    )
    e2_0 = e2_0_1 + e2_0_2
    
    lg1_a, lg1_b = None, None

    # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>
    # double excitations
    t2g_a = jnp.einsum("iajb,ia->jb", t2_aa, green_a[:nocc_a,nocc_a:]) / 4
    t2g_b = jnp.einsum("iajb,ia->jb", t2_bb, green_b[:nocc_b,nocc_b:]) / 4
    t2g_ab_a = jnp.einsum("iajb,jb->ia", t2_ab, green_b[:nocc_b,nocc_b:])
    t2g_ab_b = jnp.einsum("iajb,ia->jb", t2_ab, green_a[:nocc_a,nocc_a:])
    # t_iajb (G_ia G_jb - G_ib G_ja)
    gt2g_a = jnp.einsum("jb,jb->", t2g_a, green_a[:nocc_a,nocc_a:], 
                        optimize="optimal")
    gt2g_b = jnp.einsum("jb,jb->", t2g_b, green_b[:nocc_b,nocc_b:], 
                        optimize="optimal")
    gt2g_ab = jnp.einsum("ia,ia->", t2g_ab_a, green_a[:nocc_a,nocc_a:], 
                         optimize="optimal")
    gt2g = 2 * (gt2g_a + gt2g_b) + gt2g_ab # <exp(T1)HF|T2|walker>/<exp(T1)HF|walker>

    e1_2_1 = hg * gt2g
    
    t2_green_a = (greenp_a @ t2g_a.T) @ green_a[:nocc_a,:] # Gp_pb t_iajb G_ia G_jq
    t2_green_ab_a = (greenp_a @ t2g_ab_a.T) @ green_a[:nocc_a,:]
    t2_green_b = (greenp_b @ t2g_b.T) @ green_b[:nocc_b,:]
    t2_green_ab_b = (greenp_b @ t2g_ab_b.T) @ green_b[:nocc_b,:]
    e1_2_2_a = -jnp.einsum(
        "pq,pq->", h1_a, 4 * t2_green_a + t2_green_ab_a, optimize="optimal"
    )
    e1_2_2_b = -jnp.einsum(
        "pq,pq->", h1_b, 4 * t2_green_b + t2_green_ab_b, optimize="optimal"
    )
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2  # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>

    # e1 = e1_0 + e1_1 + e1_2

    # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>
    # double excitations
    e2_2_1 = e2_0 * gt2g
    lt2g_a = jnp.einsum("gpq,pq->g",
                        chol_a, 8 * t2_green_a + 2 * t2_green_ab_a,
                        optimize="optimal")
    lt2g_b = jnp.einsum("gpq,pq->g",
        chol_b, 8 * t2_green_b + 2 * t2_green_ab_b,
        optimize="optimal")
    e2_2_2_1 = -((lt2g_a + lt2g_b) @ (lg_a + lg_b)) / 2.0

    def scanned_fun(carry, x):
        chol_a_i, chol_b_i = x
        gl_a_i = jnp.einsum("pr,rq->pq", green_a, chol_a_i,
                            optimize="optimal")
        gl_b_i = jnp.einsum("pr,rq->pq", green_b, chol_b_i,
                            optimize="optimal")
        lt2_green_a_i = jnp.einsum(
            "pr,qr->pq", chol_a_i, 8 * t2_green_a + 2 * t2_green_ab_a,
            optimize="optimal")
        lt2_green_b_i = jnp.einsum(
            "pr,qr->pq", chol_b_i, 8 * t2_green_b + 2 * t2_green_ab_b,
            optimize="optimal")
        carry[0] += 0.5 * (
            jnp.einsum("pq,pq->", gl_a_i, lt2_green_a_i, optimize="optimal")
            + jnp.einsum("pq,pq->", gl_b_i, lt2_green_b_i, optimize="optimal")
        )
        glgp_a_i = jnp.einsum(
            "iq,qa->ia", gl_a_i[:nocc_a,:], greenp_a, optimize="optimal"
        ).astype(jnp.complex64)
        glgp_b_i = jnp.einsum(
            "iq,qa->ia", gl_b_i[:nocc_b,:], greenp_b, optimize="optimal"
        ).astype(jnp.complex64)
        l2t2_a = 0.5 * jnp.einsum(
            "ia,jb,iajb->",glgp_a_i,glgp_a_i,t2_aa.astype(jnp.float32),
            optimize="optimal")
        l2t2_b = 0.5 * jnp.einsum(
            "ia,jb,iajb->",glgp_b_i,glgp_b_i,t2_bb.astype(jnp.float32),
            optimize="optimal")
        l2t2_ab = jnp.einsum(
            "ia,jb,iajb->",glgp_a_i,glgp_b_i,t2_ab.astype(jnp.float32),
            optimize="optimal")
        carry[1] += l2t2_a + l2t2_b + l2t2_ab
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun, [0.0, 0.0], (chol_a, chol_b))
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3 # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>

    # e2 = e2_0 + e2_1 + e2_2
    o0 = jnp.linalg.det(walker_up[:nocc_a,:nocc_a]
        ) * jnp.linalg.det(walker_dn[:nocc_b,:nocc_b])
    # <exp(T1)HF|walker>/<HF|walker>
    t1 = jnp.linalg.det(wave_data["mo_ta"].T.conj() @ walker_up
        ) * jnp.linalg.det(wave_data["mo_tb"].T.conj() @ walker_dn) / o0
    t2 = gt2g * t1 # <exp(T1)HF|T2|walker>/<HF|walker>
    e0 = (e1_0 + e2_0) * t1 # <exp(T1)HF|h1+h2|walker>/<HF|walker>
    e1 = (e1_2 + e2_2) * t1 # <exp(T1)HF|T2 (h1+h2)|walker>/<HF|walker>

    return t1, t2, e0, e1

In [7]:
from jax import lax

mo_a, mo_b = wave_data['mo_ta'], wave_data['mo_tb']
chol_a = ham_data["chol"][0].reshape(-1, trial.norb, trial.norb)
chol_b = ham_data["chol"][1].reshape(-1, trial.norb, trial.norb)
green_a = (walker_up @ (jnp.linalg.inv(mo_a.T @ walker_up)) @ mo_a.T).T
green_b = (walker_dn @ (jnp.linalg.inv(mo_b.T @ walker_dn)) @ mo_b.T).T
def scanned_e2_0(carry, x):
    chol_a_i, chol_b_i = x
    lg_a_i = jnp.einsum("pr,qr->pq", chol_a_i, green_a, optimize="optimal")
    lg_b_i = jnp.einsum("pr,qr->pq", chol_b_i, green_b, optimize="optimal")
    # lg_a = jnp.einsum("gpq,pq->g", chol_a, green_a, optimize="optimal")
    # lg_b = jnp.einsum("gpq,pq->g", chol_b, green_b, optimize="optimal")
    e2_0_1_i = ((jnp.trace(lg_a_i) + jnp.trace(lg_b_i))**2) / 2.0
    e2_0_2_i = -(jnp.einsum('pq,qp->',lg_a_i,lg_a_i) 
                    + jnp.einsum('pq,qp->',lg_b_i,lg_b_i)) / 2.0
    carry[0] += e2_0_1_i
    carry[1] += e2_0_2_i

    return carry, 0.0

(e2_0_1,e2_0_2), _ = lax.scan(scanned_e2_0, [0.0,0.0], (chol_a, chol_b))
print(e2_0_1,e2_0_2)

(2.308950892026318+0j) (-1.0077269257539534+0j)


In [8]:
lg_a = jnp.einsum("gpq,pq->g", chol_a, green_a, optimize="optimal")
lg_b = jnp.einsum("gpq,pq->g", chol_b, green_b, optimize="optimal")
e2_0_1 = ((lg_a + lg_b) @ (lg_a + lg_b)) / 2.0
lg1_a = jnp.einsum("gpj,qj->gpq", chol_a, green_a, optimize="optimal")
lg1_b = jnp.einsum("gpj,qj->gpq", chol_b, green_b, optimize="optimal")
e2_0_2 = (
    -(
        jnp.sum(vmap(lambda x: x * x.T)(lg1_a))
        + jnp.sum(vmap(lambda x: x * x.T)(lg1_b))
    )
    / 2.0
)
e2_0 = e2_0_1 + e2_0_2
print(e2_0,e2_0_1,e2_0_2)

(1.3012239662723646+0j) (2.308950892026318+0j) (-1.0077269257539534+0j)


In [27]:
# @jax.jit
def _scan_e2_0(carry, x):
    chol_a_i, chol_b_i = x
    lg_a_i = jnp.einsum("pr,qr->pq", chol_a_i, green_a, optimize="optimal")
    lg_b_i = jnp.einsum("pr,qr->pq", chol_b_i, green_b, optimize="optimal")
    # lg_a = jnp.einsum("gpq,pq->g", chol_a, green_a, optimize="optimal")
    # lg_b = jnp.einsum("gpq,pq->g", chol_b, green_b, optimize="optimal")
    e2_0_1_i = (jnp.trace(lg_a_i) + jnp.trace(lg_b_i))**2 / 2.0
    e2_0_2_i = -(jnp.einsum('pq,qp->',lg_a_i,lg_a_i) 
                + jnp.einsum('pq,qp->',lg_b_i,lg_b_i)) / 2.0
    carry += e2_0_1_i + e2_0_2_i

    return carry, 0.0

# @partial(jit, static_argnums=2)
def _scan_e2_2(carry, x,trial,
               t2_green_a,t2_green_b,
               t2_green_ab_a,t2_green_ab_b,
               greenp_a,greenp_b,
               t2_aa,t2_bb,t2_ab):
    chol_a_i, chol_b_i = x
    (nocc_a,nocc_b) = trial.nelec
    gl_a_i = jnp.einsum("pr,rq->pq", green_a, chol_a_i,
                        optimize="optimal")
    gl_b_i = jnp.einsum("pr,rq->pq", green_b, chol_b_i,
                        optimize="optimal")
    lt2_green_a_i = jnp.einsum(
        "pr,qr->pq", chol_a_i, 8 * t2_green_a + 2 * t2_green_ab_a,
        optimize="optimal")
    lt2_green_b_i = jnp.einsum(
        "pr,qr->pq", chol_b_i, 8 * t2_green_b + 2 * t2_green_ab_b,
        optimize="optimal")
    carry[0] += 0.5 * (
        jnp.einsum("pq,pq->", gl_a_i, lt2_green_a_i, optimize="optimal")
        + jnp.einsum("pq,pq->", gl_b_i, lt2_green_b_i, optimize="optimal")
    )
    glgp_a_i = jnp.einsum(
        "iq,qa->ia", gl_a_i[:nocc_a,:], greenp_a, optimize="optimal"
    ).astype(jnp.complex64)
    glgp_b_i = jnp.einsum(
        "iq,qa->ia", gl_b_i[:nocc_b,:], greenp_b, optimize="optimal"
    ).astype(jnp.complex64)
    l2t2_a = 0.5 * jnp.einsum(
        "ia,jb,iajb->",glgp_a_i,glgp_a_i,t2_aa.astype(jnp.float32),
        optimize="optimal")
    l2t2_b = 0.5 * jnp.einsum(
        "ia,jb,iajb->",glgp_b_i,glgp_b_i,t2_bb.astype(jnp.float32),
        optimize="optimal")
    l2t2_ab = jnp.einsum(
        "ia,jb,iajb->",glgp_a_i,glgp_b_i,t2_ab.astype(jnp.float32),
        optimize="optimal")
    carry[1] += l2t2_a + l2t2_b + l2t2_ab
    return carry, 0.0

# e2_0, _ = lax.scan(lambda c, x:_scan_e2_0(c, x), 0.0, (chol_a, chol_b))
# print(e2_0)

In [28]:
from jax import lax

@partial(jit, static_argnums=0)
def _calc_energy_pt3(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> complex:
    nocc_a, t2_aa = trial.nelec[0], wave_data["t2aa"]
    nocc_b, t2_bb = trial.nelec[1], wave_data["t2bb"]
    t2_ab = wave_data["t2ab"]
    mo_a, mo_b = wave_data['mo_ta'], wave_data['mo_tb']
    chol_a = ham_data["chol"][0].reshape(-1, trial.norb, trial.norb)
    chol_b = ham_data["chol"][1].reshape(-1, trial.norb, trial.norb)
    h1_a = ham_data["h1"][0]
    h1_b = ham_data["h1"][1]

    # full green's function G_pq
    green_a = (walker_up @ (jnp.linalg.inv(mo_a.T @ walker_up)) @ mo_a.T).T
    green_b = (walker_dn @ (jnp.linalg.inv(mo_b.T @ walker_dn)) @ mo_b.T).T
    greenp_a = (green_a - jnp.eye(trial.norb))[:,nocc_a:]
    greenp_b = (green_b - jnp.eye(trial.norb))[:,nocc_b:]

    hg_a = jnp.einsum("pq,pq->", h1_a, green_a)
    hg_b = jnp.einsum("pq,pq->", h1_b, green_b)
    hg = hg_a + hg_b # <exp(T1)HF|h1|walker>/<exp(T1)HF|walker>

    # 0 body energy
    # h0 = ham_data["h0"]

    # <exp(T1)HF|h1|walker>/<exp(T1)HF|walker>
    # one body energy
    e1_0 = hg

    # <exp(T1)HF|h2|walker>/<exp(T1)HF|walker>
    # two body energy
    # def scanned_e2_0(carry, x):
    #     chol_a_i, chol_b_i = x
    #     lg_a_i = jnp.einsum("pr,qr->pq", chol_a_i, green_a, optimize="optimal")
    #     lg_b_i = jnp.einsum("pr,qr->pq", chol_b_i, green_b, optimize="optimal")
    #     # lg_a = jnp.einsum("gpq,pq->g", chol_a, green_a, optimize="optimal")
    #     # lg_b = jnp.einsum("gpq,pq->g", chol_b, green_b, optimize="optimal")
    #     e2_0_1_i = (jnp.trace(lg_a_i) + jnp.trace(lg_b_i))**2 / 2.0
    #     e2_0_2_i = -(jnp.einsum('pq,qp->',lg_a_i,lg_a_i) 
    #                  + jnp.einsum('pq,qp->',lg_b_i,lg_b_i)) / 2.0
    #     carry += e2_0_1_i + e2_0_2_i

    #     return carry, 0.0
    
    # e2_0, _ = lax.scan(scanned_e2_0, 0.0, (chol_a, chol_b))
    e2_0, _ = lax.scan(lambda c, x:_scan_e2_0(c, x), 0.0, (chol_a, chol_b))

    # e2_0 = e2_0_1 + e2_0_2
    # lg1_a, lg1_b = None, None

    # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>
    # double excitations
    t2g_a = jnp.einsum("iajb,ia->jb", t2_aa, green_a[:nocc_a,nocc_a:]) / 4
    t2g_b = jnp.einsum("iajb,ia->jb", t2_bb, green_b[:nocc_b,nocc_b:]) / 4
    t2g_ab_a = jnp.einsum("iajb,jb->ia", t2_ab, green_b[:nocc_b,nocc_b:])
    t2g_ab_b = jnp.einsum("iajb,ia->jb", t2_ab, green_a[:nocc_a,nocc_a:])
    # t_iajb (G_ia G_jb - G_ib G_ja)
    gt2g_a = jnp.einsum("jb,jb->", t2g_a, green_a[:nocc_a,nocc_a:], 
                        optimize="optimal")
    gt2g_b = jnp.einsum("jb,jb->", t2g_b, green_b[:nocc_b,nocc_b:], 
                        optimize="optimal")
    gt2g_ab = jnp.einsum("ia,ia->", t2g_ab_a, green_a[:nocc_a,nocc_a:], 
                         optimize="optimal")
    gt2g = 2 * (gt2g_a + gt2g_b) + gt2g_ab # <exp(T1)HF|T2|walker>/<exp(T1)HF|walker>

    e1_2_1 = hg * gt2g
    
    t2_green_a = (greenp_a @ t2g_a.T) @ green_a[:nocc_a,:] # Gp_pb t_iajb G_ia G_jq
    t2_green_ab_a = (greenp_a @ t2g_ab_a.T) @ green_a[:nocc_a,:]
    t2_green_b = (greenp_b @ t2g_b.T) @ green_b[:nocc_b,:]
    t2_green_ab_b = (greenp_b @ t2g_ab_b.T) @ green_b[:nocc_b,:]
    e1_2_2_a = -jnp.einsum(
        "pq,pq->", h1_a, 4 * t2_green_a + t2_green_ab_a, optimize="optimal"
    )
    e1_2_2_b = -jnp.einsum(
        "pq,pq->", h1_b, 4 * t2_green_b + t2_green_ab_b, optimize="optimal"
    )
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2  # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>

    # e1 = e1_0 + e1_1 + e1_2

    # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>
    # double excitations
    e2_2_1 = e2_0 * gt2g
    lg_a = jnp.einsum("gpq,pq->g", chol_a, green_a, optimize="optimal")
    lg_b = jnp.einsum("gpq,pq->g", chol_b, green_b, optimize="optimal")
    lt2g_a = jnp.einsum("gpq,pq->g",
                        chol_a, 8 * t2_green_a + 2 * t2_green_ab_a,
                        optimize="optimal")
    lt2g_b = jnp.einsum("gpq,pq->g",
        chol_b, 8 * t2_green_b + 2 * t2_green_ab_b,
        optimize="optimal")
    e2_2_2_1 = -((lt2g_a + lt2g_b) @ (lg_a + lg_b)) / 2.0

    # def scanned_fun(carry, x):
    #     chol_a_i, chol_b_i = x
    #     gl_a_i = jnp.einsum("pr,rq->pq", green_a, chol_a_i,
    #                         optimize="optimal")
    #     gl_b_i = jnp.einsum("pr,rq->pq", green_b, chol_b_i,
    #                         optimize="optimal")
    #     lt2_green_a_i = jnp.einsum(
    #         "pr,qr->pq", chol_a_i, 8 * t2_green_a + 2 * t2_green_ab_a,
    #         optimize="optimal")
    #     lt2_green_b_i = jnp.einsum(
    #         "pr,qr->pq", chol_b_i, 8 * t2_green_b + 2 * t2_green_ab_b,
    #         optimize="optimal")
    #     carry[0] += 0.5 * (
    #         jnp.einsum("pq,pq->", gl_a_i, lt2_green_a_i, optimize="optimal")
    #         + jnp.einsum("pq,pq->", gl_b_i, lt2_green_b_i, optimize="optimal")
    #     )
    #     glgp_a_i = jnp.einsum(
    #         "iq,qa->ia", gl_a_i[:nocc_a,:], greenp_a, optimize="optimal"
    #     ).astype(jnp.complex64)
    #     glgp_b_i = jnp.einsum(
    #         "iq,qa->ia", gl_b_i[:nocc_b,:], greenp_b, optimize="optimal"
    #     ).astype(jnp.complex64)
    #     l2t2_a = 0.5 * jnp.einsum(
    #         "ia,jb,iajb->",glgp_a_i,glgp_a_i,t2_aa.astype(jnp.float32),
    #         optimize="optimal")
    #     l2t2_b = 0.5 * jnp.einsum(
    #         "ia,jb,iajb->",glgp_b_i,glgp_b_i,t2_bb.astype(jnp.float32),
    #         optimize="optimal")
    #     l2t2_ab = jnp.einsum(
    #         "ia,jb,iajb->",glgp_a_i,glgp_b_i,t2_ab.astype(jnp.float32),
    #         optimize="optimal")
    #     carry[1] += l2t2_a + l2t2_b + l2t2_ab
    #     return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(
        lambda c, x: _scan_e2_2(c,x,trial,
                                t2_green_a,t2_green_b,
                                t2_green_ab_a,t2_green_ab_b,
                                greenp_a,greenp_b,
                                t2_aa,t2_bb,t2_ab), [0.0, 0.0], (chol_a, chol_b))
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3 # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>

    o0 = jnp.linalg.det(walker_up[:nocc_a,:nocc_a]
        ) * jnp.linalg.det(walker_dn[:nocc_b,:nocc_b])
    # <exp(T1)HF|walker>/<HF|walker>
    t1 = jnp.linalg.det(wave_data["mo_ta"].T.conj() @ walker_up
        ) * jnp.linalg.det(wave_data["mo_tb"].T.conj() @ walker_dn) / o0
    t2 = gt2g * t1 # <exp(T1)HF|T2|walker>/<HF|walker>
    e0 = (e1_0 + e2_0) * t1 # <exp(T1)HF|h1+h2|walker>/<HF|walker>
    e1 = (e1_2 + e2_2) * t1 # <exp(T1)HF|T2 (h1+h2)|walker>/<HF|walker>

    return t1, t2, e0, e1

In [45]:
mo_ta = wave_data['mo_ta']
mo_tb = wave_data['mo_tb']
(noccA, noccB) = trial.nelec
wave_data["rot_t2aa"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_ta[:noccA,:noccA].T,mo_ta[:noccA,:noccA].T,t2aa)
wave_data["rot_t2bb"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_tb[:noccB,:noccB].T,mo_tb[:noccB,:noccB].T,t2bb)
wave_data["rot_t2ab"] = jnp.einsum('ik,jl,kalb->iajb',
    mo_ta[:noccA,:noccA].T,mo_tb[:noccB,:noccB].T,t2ab)

ham_data["rot_h1_t"] = [
    wave_data["mo_ta"].T.conj() @ ham_data["h1"][0],
    wave_data["mo_tb"].T.conj() @ ham_data["h1"][1],
]
ham_data["rot_chol_t"] = [
    jnp.einsum(
        "ip,gpq->giq",
        wave_data["mo_ta"].T.conj(),
        ham_data["chol"][0].reshape(-1, trial.norb, trial.norb),
    ),
    jnp.einsum(
        "ip,gpq->giq",
        wave_data["mo_tb"].T.conj(),
        ham_data["chol"][1].reshape(-1, trial.norb, trial.norb),
    ),
]

In [46]:
from jax import lax
@partial(jit, static_argnums=0)
def _calc_energy_pt2(
    trial,
    walker_up: jax.Array,
    walker_dn: jax.Array,
    ham_data: dict,
    wave_data: dict,
) -> complex:
    nocc_a, rot_t2aa = trial.nelec[0], wave_data["rot_t2aa"]
    nocc_b, rot_t2bb = trial.nelec[1], wave_data["rot_t2bb"]
    rot_t2ab = wave_data["rot_t2ab"]
    mo_a, mo_b = wave_data['mo_ta'], wave_data['mo_tb']
    chol_a = ham_data["chol"][0].reshape(-1, trial.norb, trial.norb)
    chol_b = ham_data["chol"][1].reshape(-1, trial.norb, trial.norb)
    h1_a, h1_b = ham_data["h1"][0], ham_data["h1"][1]
    rot_h1a, rot_h1b = ham_data["rot_h1_t"][0], ham_data["rot_h1_t"][1]
    rot_chol_a, rot_chol_b = ham_data["rot_chol_t"][0], ham_data["rot_chol_t"][1]

    # rotated green's function rot_G_pl = phi_pk inv(psi^dagger psi)_kl
    rot_ga = (walker_up @ (jnp.linalg.inv(mo_a.T @ walker_up)))
    rot_gb = (walker_dn @ (jnp.linalg.inv(mo_b.T @ walker_dn)))

    # h1g_a = jnp.einsum("pl,lp->", rot_ga, rot_h1a)
    gh1_a = jnp.einsum("ql,lp->qp",rot_ga,rot_h1a)
    gh1_b = jnp.einsum("ql,lp->qp",rot_gb,rot_h1b)
    e1_0 = jnp.trace(gh1_a + gh1_b) # <exp(T1)HF|h1|walker>/<exp(T1)HF|walker>

    # <exp(T1)HF|h2|walker>/<exp(T1)HF|walker>
    gl_a = jnp.einsum("pl,glq->gpq",rot_ga,rot_chol_a,optimize="optimal")
    gl_b = jnp.einsum("pl,glq->gpq",rot_gb,rot_chol_b,optimize="optimal")
    c_a = vmap(jnp.trace)(gl_a)
    c_b = vmap(jnp.trace)(gl_b)
    e2_0_1 = jnp.sum(c_a * c_a) \
             + jnp.sum(c_b * c_b)
    e2_0_2 = jnp.sum(vmap(lambda x: x * x.T)(gl_a)) \
             + jnp.sum(vmap(lambda x: x * x.T)(gl_b))
    e2_0 = e2_0_1 + e2_0_2

    # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>
    # double excitations
    t2g_a = jnp.einsum("ai,iajb->jb", rot_ga[nocc_a:,:], rot_t2aa, 
                       optimize="optimal") / 4
    t2g_b = jnp.einsum("ai,iajb->jb", rot_gb[nocc_b:,:], rot_t2bb,
                       optimize="optimal") / 4
    t2g_ab_a = jnp.einsum("iajb,bj->ia", rot_t2ab, rot_gb[nocc_b:,:],
                          optimize="optimal")
    t2g_ab_b = jnp.einsum("iajb,bj->jb", rot_t2ab, rot_ga[nocc_a:,:],
                          optimize="optimal")
    # t_iajb (G_ia G_jb - G_ib G_ja)
    gt2g_a = jnp.einsum("qj,jb->qb",rot_ga,t2g_a,optimize="optimal")
    # gt2g_a = jnp.einsum("jb,bj->", t2g_a, rot_ga[nocc_a:,:], 
    #                     optimize="optimal")
    # gt2g_b = jnp.einsum("jb,bj->", t2g_b, rot_gb[nocc_b:,:], 
    #                     optimize="optimal")
    gt2g_b = jnp.einsum("qj,jb->qb",rot_gb,t2g_b,optimize="optimal")
    gt2g_ab = jnp.einsum("ai,ia->", rot_ga[nocc_a:,:], t2g_ab_a,
                         optimize="optimal")
    gt2g = 2 * jnp.trace(gt2g_a[nocc_a:,:] + gt2g_b[nocc_b:,:]) + gt2g_ab # <exp(T1)HF|T2|walker>/<exp(T1)HF|walker>

    e1_2_1 = e1_0 * gt2g
    
    # gp_bp h1_pq = g_bp h1_pq - h1_bq 
    gph1_a = gh1_a[nocc_a:,:] - h1_a[nocc_a:,:]
    gph1_b = gh1_b[nocc_b:,:] - h1_b[nocc_b:,:]
    # G_qj t_iajb G_ia
    gt2g_ab_a = jnp.einsum("qj,jb->qb",rot_ga,t2g_ab_a,optimize="optimal")
    gt2g_ab_b = jnp.einsum("qj,jb->qb",rot_gb,t2g_ab_b,optimize="optimal")
    e1_2_2_a = - jnp.einsum("pb,bq->", 4 * gt2g_a + gt2g_ab_a, gph1_a,
                            optimize="optimal")
    e1_2_2_b = - jnp.einsum("pb,bq->", 4 * gt2g_b + gt2g_ab_b, gph1_b,
                            optimize="optimal")
    e1_2_2 = e1_2_2_a + e1_2_2_b
    e1_2 = e1_2_1 + e1_2_2  # <exp(T1)HF|T2 h1|walker>/<exp(T1)HF|walker>

    # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>
    e2_2_1 = e2_0 * gt2g
    # gl_a = jnp.einsum("pl,glq->gpq",rot_ga,rot_chol_a,optimize="optimal")
    # gl_b = jnp.einsum("pl,glq->gpq",rot_gb,rot_chol_b,optimize="optimal")
    gpl_a = gl_a[:,nocc_a:,:] - chol_a[:,nocc_a:,:]
    gpl_b = gl_a[:,nocc_a:,:] - chol_b[:,nocc_b:,:]
    lt2g_a = jnp.einsum("gbq,qb->g",gpl_a, 8 * gt2g_a + 2 * gt2g_ab_a,
                        optimize="optimal")
    lt2g_b = jnp.einsum("gbq,qb->g",gpl_b, 8 * gt2g_b + 2 * gt2g_ab_b,
        optimize="optimal")
    e2_2_2_1 = -((lt2g_a + lt2g_b) @ (c_a + c_b)) / 2.0

    def scanned_fun(carry, x):
        gl_a_i,gl_b_i,gpl_a_i,gpl_b_i = x
        glgt2g_a_i = jnp.einsum("sr,rb->sb", gl_a_i, gt2g_a) # L_pr G_ps t_iajb G_ai G_jr
        glgt2g_b_i = jnp.einsum("sr,rb->sb", gl_b_i, gt2g_b)
        carry[0] += 0.5 * (
            jnp.einsum("bs,sb->", gpl_a_i, glgt2g_a_i, optimize="optimal")
            + jnp.einsum("bs,sb->", gpl_b_i, glgt2g_b_i, optimize="optimal")
        )
        gplg_a_i = jnp.einsum("bs,sj->bj", gpl_a_i,rot_ga,
                              optimize="optimal")#.astype(jnp.complex64)
        gplg_b_i = jnp.einsum("bs,sj->bj", gpl_b_i,rot_gb,
                              optimize="optimal")#.astype(jnp.complex64)
        l2t2_a = 0.5 * jnp.einsum("ai,iajb,bj->",gplg_a_i,rot_t2aa,gplg_a_i,
                                  optimize="optimal")
        l2t2_b = 0.5 * jnp.einsum("ai,iajb,bj->",gplg_b_i,rot_t2bb,gplg_b_i,
                                  optimize="optimal")
        l2t2_ab = jnp.einsum("ai,iajb,bj->",gplg_a_i,rot_t2ab,gplg_b_i,
                             optimize="optimal")
        carry[1] += l2t2_a + l2t2_b + l2t2_ab
        return carry, 0.0

    [e2_2_2_2, e2_2_3], _ = lax.scan(scanned_fun,[0.0, 0.0],(gl_a,gl_b,gpl_a,gpl_b))
    e2_2_2 = e2_2_2_1 + e2_2_2_2
    e2_2 = e2_2_1 + e2_2_2 + e2_2_3 # <exp(T1)HF|T2 h2|walker>/<exp(T1)HF|walker>

    # e2 = e2_0 + e2_1 + e2_2
    o0 = jnp.linalg.det(walker_up[:nocc_a,:nocc_a]
        ) * jnp.linalg.det(walker_dn[:nocc_b,:nocc_b])
    # <exp(T1)HF|walker>/<HF|walker>
    t1 = jnp.linalg.det(wave_data["mo_ta"].T.conj() @ walker_up
        ) * jnp.linalg.det(wave_data["mo_tb"].T.conj() @ walker_dn) / o0
    t2 = gt2g * t1 # <exp(T1)HF|T2|walker>/<HF|walker>
    e0 = (e1_0 + e2_0) * t1 # <exp(T1)HF|h1+h2|walker>/<HF|walker>
    e1 = (e1_2 + e2_2) * t1 # <exp(T1)HF|T2 (h1+h2)|walker>/<HF|walker>

    return t1, t2, e0, e1

In [12]:
# mycc.t1 = (mycc.t1[0]*0, mycc.t1[1]*0)
eris = mycc.ao2mo(mycc.mo_coeff)
eccs = mycc.energy(mycc.t1,(0*mycc.t2[0],0*mycc.t2[1],0*mycc.t2[2]), eris)
eccs = mf.e_tot + eccs
print('ccs energy: ', eccs)
eccsd = mycc.energy(mycc.t1, mycc.t2, eris)
eccsd = mf.e_tot + eccsd
print('ccsd energy: ', eccsd)

ccs energy:  -1.5441982437908104
ccsd energy:  -1.5733064210885241


In [32]:
h0 = ham_data['h0']
walker_up, walker_dn = prop_data['walkers'][0][0], prop_data['walkers'][1][0]
import time
start = time.perf_counter()
t1,t2,e0,e1 = _calc_energy_pt3(trial, walker_up, walker_dn, ham_data, wave_data)
end = time.perf_counter()
print(t1,t2,e0,e1)
print(h0 + 1/t1 * e0 )
print(h0 + 1/t1 * e0 - eccs)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0 - eccsd)
print(end-start)

(-0.9977535395017092+0j) (-0+0j) (2.787921182166474-0j) (0.02904278935720947-0j)
(-1.5441982381328332+0j)
(5.657977197870423e-09+0j)
(-1.5733064178659903+0j)
(3.2225337953661892e-09+0j)
0.00048703799984650686


In [22]:
h0 = ham_data['h0']
walker_up, walker_dn = prop_data['walkers'][0][0], prop_data['walkers'][1][0]
import time
start = time.perf_counter()
t1,t2,e0,e1 = _calc_energy_pt(trial, walker_up, walker_dn, ham_data, wave_data)
end = time.perf_counter()
print(t1,t2,e0,e1)
print(h0 + 1/t1 * e0 )
print(h0 + 1/t1 * e0 - eccs)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)
print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0 - eccsd)
print(end-start)

NameError: name '_calc_energy_pt' is not defined

In [51]:
start = time.perf_counter()
e = trial._calc_energy(walker_up, walker_dn, ham_data, wave_data)
end = time.perf_counter()
print(e)
# print(h0 + 1/t1 * e0 )
# print(h0 + 1/t1 * e0 - eccs)
# print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0)
# print(h0 + 1/t1 * e0 + 1/t1 * e1 - 1/t1**2 * t2 * e0 - eccsd)
print(end-start)

(-1.5439139532327752+0j)
0.0005974590021651238
